### Quality control script 

#### - Remove data for dead or abnormal larvae 
#### - Remove incomplete data 
#### - Convert excel files to more efficient .pkl files for future processing.

by: Dr. Adrian Green, *Fall 2020*

##### Early data (Nov_2019 - March_2020) used the original 18 morphology endpoints. Additional data (Dec_2020) used super-endpoints and was kept aside as validation data.

In [1]:
# import libraries

import numpy as np
import pandas as pd
import openpyxl
import os
import xlrd 
import sys

from Green_scripts_v1 import save_obj

basedirname = '/home2/ajgreen4/ZF_Projects/HTS_Behavior_Project'

In [2]:
# Load identification data

# Read in well to arena key
waKey = pd.read_csv(basedirname + "/Raw_Data_Files/well-arena.csv")
[waNumRows, waNumCols] = waKey.shape

# Read in plate to chemical to well key
pcwKey = pd.read_csv(basedirname + "/Raw_Data_Files/plateTable.csv")
[pcwNumRows, pcwNumCols] = pcwKey.shape


### Training data (super-endpoints)

In [3]:
# Define location to look for plateID
timecheck = 0 # set to 1 to output a file with filename, Trial duration, and Recording duration

# output location
dataPath = basedirname + "/QC_Data_Files/OSU_Train_super_endpoints/"

#Validation Files
rawDataPath = basedirname + "/Raw_Data_Files/OSU_Train_super_endpoints/OSU_2020Dec_raw/"

newlist = os.listdir(rawDataPath)
mylist = []
for names in newlist:
    if names.endswith(".xlsx"):
        mylist.append(names)
print("Number of raw data files: ",len(mylist))

# Read in superendpoints morphology data - Validation
morphology_df = pd.read_csv(basedirname + "/Raw_Data_Files/morphology_super.csv")
morphology_df = morphology_df.fillna(0)

Number of raw data files:  8


#### Code to reads in one ethovision raw export file at a time, delete sheets that don't meet QC requirments and write cleaned excel files to disk.
##### Experinental design : 6 minutes acclimation then 18 minutes of light/dark cycling (3 x 2 x 3 min)
Initial QC need to include the second light cycle therefore 720s to 1080s

In [4]:
%%time
try: 
    os.chdir(dataPath) 
# Caching the exception     
except: 
    os.makedirs(dataPath)   

i=j=0
for file in mylist:
    filePath = rawDataPath + file

    # for an earlier version of Excel, you may need to use the file extension of 'xls'
#     noldusDataFile = pd.read_excel(filePath, sheet_name=None, header=None, usecols='A:D,H,I')
    noldusDataFile = pd.read_excel(filePath, sheet_name=None, header=None, usecols=None)

    etho_data_df = []
    try:
        for key, sheet in noldusDataFile.items():
            locData = pd.DataFrame(sheet)
            headerLines = int(locData.loc[0, 1])
            [numRows, numCols] = locData.shape
            # make sure that light cycle data needed are present
            if numRows > 35 and int(locData.loc[headerLines+1,0]) <= 720 and int(locData.loc[numRows-1,0]) >= 1080:
                plateID = ""
                chemID = ""
                well = ""

                try:
                    plateID = int(os.path.splitext(file)[0][0:5])
                except:  
                    try:
                        end = len(os.path.splitext(file)[0])
                        plateID = int(os.path.splitext(file)[0][end-5:end])
                    except:
                        aviName = locData.loc[(locData[0] == 'Video file')].iloc[0,1]
                        end = len(aviName)-4
                        plateID = int(aviName[end-5:end])
                arenaName = int(locData.loc[(locData[0] == 'Arena name')].iloc[0,1])
                well = waKey.loc[(waKey['arena']==arenaName)].iloc[0,0]
                try:
                    chemID = pcwKey.loc[(pcwKey['PlateID']==plateID) & (pcwKey['Well']==well)].iloc[0,1]
                    conc = pcwKey.loc[(pcwKey['PlateID']==plateID) & (pcwKey['Well']==well)].iloc[0,3]

    #                 print("Plate:", plateID, " Chem:", chemID, " Well:", well)
                    # get morphology data for lavae
                    subset_df = morphology_df[(morphology_df.iloc[:,3]==int(plateID)) & 
                                              (morphology_df.iloc[:,4]==well) &
                                              (morphology_df.iloc[:,0]==int(chemID))].iloc[:,6:28].T
                    # if no abnormalities then keep record
                    if int(subset_df.max()):
                        etho_data_df.append(key)
                except:
                    # remove sheets with no chemical info
                    etho_data_df.append(key)
            else:
                # remove sheets with less than 18 minutes of light cycle data
                etho_data_df.append(key)
        for key in etho_data_df:
            del noldusDataFile[key]

        if len(noldusDataFile): # save as pickkle to maintain 3d data structure
            save_obj(noldusDataFile, dataPath, os.path.splitext(file)[0])

        print(len(etho_data_df),' wells removed from ', file)
    except Exception as e:
        print('Error in : ', file, " Plate:", plateID, " Chem:", chemID, " Well:", well)
        print(e)

File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21549 .pkl
21  wells removed from  Raw data-21549.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21548 .pkl
19  wells removed from  Raw data-21548.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21547 .pkl
15  wells removed from  Raw data-21547.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21546 .pkl
22  wells removed from  Raw data-21546.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21543 .pkl
26  wells removed from  Raw data-21543.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_super_endpoints/ Raw data-21542 .pkl
28  wells removed from  Raw data-21542.xls

### Training data (18 endpoints)

In [5]:
# Define location to look for plateID
timecheck = 0 # set to 1 to output a file with filename, Trial duration, and Recording duration

# output location
dataPath = basedirname + "/QC_Data_Files/OSU_Train_original_endpoints/"
#Noldus raw data file location
# Training Files
rawDataPath = basedirname + "/Raw_Data_Files/OSU_Train_original_endpoints/"

sublist = os.listdir(rawDataPath)
mylist = []
for subdir in sublist:
    newlist = os.listdir(rawDataPath+subdir+'/')
    for names in newlist:
        if names.endswith(".xlsx"):
            mylist.append(subdir + '/' + names)
print("Number of raw data files: ",len(mylist))

# Read in morphology data - Training
morphology_df = pd.read_csv(basedirname + "/Raw_Data_Files/morphology.csv")
morphology_df = morphology_df.fillna(0)


Number of raw data files:  41


#### Code to reads in one ethovision raw export file at a time, delete sheets that don't meet QC requirments and write cleaned excel files to disk.
##### Experinental design : 6 minutes acclimation then 18 minutes of light/dark cycling (3 x 2 x 3 min)
Initial QC need to include the second light cycle therefore 720s to 1080s

In [6]:
%%time
try: 
    os.chdir(dataPath) 
# Caching the exception     
except: 
    os.makedirs(dataPath)   

i=j=0
for file in mylist:
    filePath = rawDataPath + file

    # for an earlier version of Excel, you may need to use the file extension of 'xls'
#     noldusDataFile = pd.read_excel(filePath, sheet_name=None, header=None, usecols='A:D,H,I')
    noldusDataFile = pd.read_excel(filePath, sheet_name=None, header=None, usecols=None)

    etho_data_df = []
    try:
        for key, sheet in noldusDataFile.items():
            locData = pd.DataFrame(sheet)
            headerLines = int(locData.loc[0, 1])
            [numRows, numCols] = locData.shape
            # make sure that light cycle data needed are present
            if numRows > 35 and int(locData.loc[headerLines+1,0]) <= 720 and int(locData.loc[numRows-1,0]) >= 1080:
                plateID = ""
                chemID = ""
                well = ""

                try:
                    plateID = int(os.path.split(file)[1][0:5])
                except:  
                    try:
                        end = len(os.path.split(file)[1])
                        plateID = int(os.path.split(file)[1][end-5:end])
                    except:
                        aviName = locData.loc[(locData[0] == 'Video file')].iloc[0,1]
                        end = len(aviName)-4
                        plateID = int(aviName[end-5:end])
                arenaName = int(locData.loc[(locData[0] == 'Arena name')].iloc[0,1])
                well = waKey.loc[(waKey['arena']==arenaName)].iloc[0,0]
                try:
                    chemID = pcwKey.loc[(pcwKey['PlateID']==plateID) & (pcwKey['Well']==well)].iloc[0,1]
                    conc = pcwKey.loc[(pcwKey['PlateID']==plateID) & (pcwKey['Well']==well)].iloc[0,3]

    #                 print("Plate:", plateID, " Chem:", chemID, " Well:", well)
                    # get morphology data for lavae
                    subset_df = morphology_df[(morphology_df.iloc[:,3]==int(plateID)) & 
                                              (morphology_df.iloc[:,4]==well) &
                                              (morphology_df.iloc[:,0]==int(chemID))].iloc[:,6:28].T
                    # if no abnormalities then keep record
                    if int(subset_df.max()):
                        etho_data_df.append(key)
                except:
                    # remove sheets with no chemical info
                    etho_data_df.append(key)
            else:
                # remove sheets with less than 18 minutes of light cycle data
                etho_data_df.append(key)
        for key in etho_data_df:
            del noldusDataFile[key]

        if len(noldusDataFile): # save as pickkle to maintain 3d data structure
            save_obj(noldusDataFile, dataPath, os.path.splitext(os.path.basename(file))[0])
            
        print(len(etho_data_df),' wells removed from ', file)
    except Exception as e:
        print('Error in : ', file, " Plate:", plateID, " Chem:", chemID, " Well:", well)
        print(e)

File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_original_endpoints/ 19861_Raw data .pkl
25  wells removed from  OSU_2020Feb_raw/19861_Raw data.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_original_endpoints/ 19840-Raw data .pkl
9  wells removed from  OSU_2020Feb_raw/19840-Raw data.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_original_endpoints/ 19839-Raw data .pkl
44  wells removed from  OSU_2020Feb_raw/19839-Raw data.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_original_endpoints/ 19833_Raw data .pkl
9  wells removed from  OSU_2020Feb_raw/19833_Raw data.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Files/OSU_Train_original_endpoints/ 19832_Raw data .pkl
24  wells removed from  OSU_2020Feb_raw/19832_Raw data.xlsx
File saved:  /home2/ajgreen4/ZF_Projects/HTS_Behavior_Project/QC_Data_Fi